# Data Badge: Data Sources


### - In this notebook:

- [Reading local CSV files](#local-files)
- [Reading CSV files from a URL](#reading-url)
- [Reading JSON files](#reading-json)
- [Reading from database files](#reading-databases)
- [Making API calls](#api-calls)
- Mini-diary ⭐️⭐️⭐️❓


In [ ]:
import pandas as pd
import numpy as np
import json

<a id="local-files"></a>
# 1. Reading Local CSV Files

#### What are CSV (comma separated values) files?

These basically look like simplified Excel. First line of the file contains names of columns, and then each next line is a set of values for those columns. Values are separated by commas.

`
name, age, department
Pim, 34, ER
Jannet, 54, Oncology
Aoife, 25, Oncology
`

Notice you cannot skip any values, since the only indicator of what data means is in which order it is. So for example, if you do not know one person's name you can't just write:

```Natasha, ER```

...becuase it would treat ER as their age. (second value means second column). Instead sometimes you will see missing data just skipped with a missing value (nothing between commas) like this:

```Natasha,,ER```

Local files are files that are on your computer, often stored in a 'data' folder.

In [ ]:
# Let's import some data from a local CSV file using Pandas.

# If we did not specify index_col, index would be 0,1,2,3,... but this data already has a good index 
nursing_home_residents = pd.read_csv("./data/nursing_home_residents.csv")
nursing_home_residents

In [ ]:
# And we could create an index, but in this dataset none of the columns is uniquely identifying a row
# Instead what is unique is the combination of date, statistic and CA (council area)

# So E.G. If the date was a unique row item, we could use index_col to specify index
# nursing_home_residents = pd.read_csv("./data/nursing_home_residents.csv", index_col='Date')
# nursing_home_residents

<a id="reading-url"></a>
# 2. Reading CSV Files from a URL

### Online files use EXACTLY the same code

This is just so amazing. If the 'filename' you use in read_csv starts with http... it will be simply loaded from the internet.

In [ ]:
nursing_home_residents = pd.read_csv("https://www.opendata.nhs.scot/dataset/75cca0a9-780d-40e0-9e1f-5f4796950794/resource/139f61d8-a87d-419d-b7af-31f555a60c89/download/file3_mean_median_age_years.csv")
nursing_home_residents

<a id="reading-json"></a>
# 3. Reading JSON Files

#### What are JSON files? (key-value pairs, like Python dictionary)

These will be very familiar to you. Lists are indicated by [ ] and key value pairs (separated by : colon) are enclosed in { }

```
[
    {'name':'Pim', 'age':34, 'department':'Oncology'},
    {'name':'Jannet', 'age':54, 'department':'Oncology'},
    {'name':'Aoife', 'age':25, 'department':'Oncology'}
]
```

Sometimes JSON is simple but other times it can be complex (e.g. very nested (things, within things, within things))

In [ ]:
simple_staff = pd.read_json("./data/simple.json")
simple_staff

But if the file is 'nested' as in, it contains things within things, we need to tell Pandas how we want it read.
Notice that in file `nested_deeper.json` the staff members are no longer at the top level. Also that there is a List of information ('shifts') inside of each staff member. It is no longer obvious how to change it into an Excel spreadsheet format that is so easy for CSV.

We will need to guide Pandas deeper into the datastructure... and say I care about: `inside of hospitals, the staff collection`.

```
{
  "requested_date": "01012025",
  "server_info": {"location": "europe-2", "timezone": "GMT"},
  "hospitals": [
    {
      "hospital_name":"Western General",
      "location":{"city":"Edinburgh", "board":"NHS Lothian"},
      "staff":[
        {"name":"Pim", "age":34, "department":"ER", "shifts": ["day"]},
        {"name":"Jannet", "age":54, "department":"Oncology", "shifts": ["day", "night"]},
        {"name":"Aoife", "age":25, "department":"Oncology",  "shifts": ["night"]}
      ]
    },
    {
      "hospital_name":"Raigmore Hospital",
      "location":{"city":"Inverness", "board":"NHS Highland Board"},
      "staff":[
        {"name":"Pat", "age":24, "department":"ER", "shifts": ["day"]},
        {"name":"Jimmy", "age":44, "department":"ER", "shifts": ["day", "night"]},
        {"name":"Siobhan", "age":45, "department":"ER",  "shifts": ["day", "night"]}
      ]
    }
  ]
  
}
```

If we do not specify what we mean, we'll get something like this: (not very useful)

In [ ]:
# nested_staff = pd.read_json("./data/nested_deeper.json")
# nested_staff

## this will throw an error because Panda doesn;t know part is important (staff!)
## ValueError: Mixing dicts with non-Series may lead to ambiguous ordering. 
## we commented this out as the error is massive :D Uncomment and have a look, but then comment it back out.

So we need to specify things:

- in data jump into the `hospitals`, and then start parsing
- staff members are nested under key `staff`
- then inside each staff member I want all of their data, but also some 'meta information'. see code below and try to figure out what it does.

What we do here is:

1. We Load the file, and then create another dataframe by 'interpreting' just one of the columns items (`hospitals`), then each item in the sub-collection the `staff`

This is a simple way if your data is simple:

In [ ]:
with open("./data/nested_deeper.json", 'r') as file:
    nested_all_data = json.load(file)  # Load the JSON data from the file

# Normalize the 'staff' array into a DataFrame
df_staff = pd.json_normalize(
    nested_all_data['hospitals'],
    record_path=["staff"], 
    meta= ["hospital_name", ["location", "city"]] )
df_staff
# bracket-usage challange: how would you add the info of the "board" as well?

Notice the `shifts` column has data as lists. We can use `explode()` to expand the shifts column into separate rows. Here's how:

In [ ]:
# Explode the 'shifts' column. This will make the data long (a bit). There's another way, see below.
df_staff_long = df_staff.explode("shifts")
df_staff_long

And if you prefer dummy variables (useful for some visualisations and machine learning tasks)

In [ ]:
# turn data into binarry dummies: 1 if present, 0 if not-present 
dummies = pd.get_dummies(df_staff['shifts'].explode()).groupby(level=0).sum()
dummies = dummies.add_prefix('shift.')
shifts_df_dummies = pd.concat([dummies, df_staff], axis = 1)
shifts_df_dummies


There are more examples of normalising and converting nested JSON <a href="https://pandas.pydata.org/docs/dev/reference/api/pandas.json_normalize.html">here</a> and <a href="https://saturncloud.io/blog/how-to-convert-nested-json-to-pandas-dataframe-with-specific-format/#converting-nested-json-to-pandas-dataframe">here</a>

<a id="reading-databases"></a>
# 4. Reading from Database files

### A Quick Explanation of Relational Databases

Relational databases organise data into tables with rows and columns, like interconnected spreadsheets.

Each table represents a specific entity and relationships between tables are established using primary and foreign keys, ensuring data integrity and reducing redundancy.

SQL is the standard language used to interact with these databases, allowing for efficient querying, manipulation, and management of large, complex datasets, making them essential for structured data storage and analysis.

Here we will use a **different flavour of sql** than in the video. This flavour, called `sqlite3` is simpler when we are working on **local db file** (rather than an online database which we connect to using the internet).

Notice we use a `cursor` which is the **active connection to the database**. You can think of a cursor as a mouse pointer on your computer, or a needle on a old vinyl record - it is pointing at the database, and you can give commands to it.

In [81]:
import sqlite3

# Connect to db file called my_example_db.sqlite (will create it if file it doens't exist)
connection = sqlite3.connect("./data/students_pets_db.sqlite")
cursor = connection.cursor()

In [82]:
# This will delete all database data you created before! 
cursor.execute("DROP TABLE IF EXISTS Levels;")
cursor.execute("DROP TABLE IF EXISTS Students;")
cursor.execute("DROP TABLE IF EXISTS Pets;")
cursor.execute("DROP TABLE IF EXISTS Owners;")

In [83]:
# for now the database is empty, so let's create some tables 
# Levels
create_levels_sql = """
CREATE TABLE IF NOT EXISTS Levels (
    id INTEGER PRIMARY KEY,
    name TEXT,
    short_name TEXT)
"""
cursor.execute(create_levels_sql)

# Students
create_students_sql = """
CREATE TABLE IF NOT EXISTS Students (
    id INTEGER PRIMARY KEY,
    name TEXT,
    age INTEGER,
    level_id INTEGER)
"""
cursor.execute(create_students_sql)

# Pets
create_pets_sql = """
CREATE TABLE IF NOT EXISTS Pets (
    id INTEGER PRIMARY KEY,
    name TEXT)
"""
cursor.execute(create_pets_sql)

# Owners
create_owners_sql = """
CREATE TABLE IF NOT EXISTS Owners (
    id INTEGER PRIMARY KEY,
    student_id INTEGER,
    pet_id INTEGER
    )
"""
cursor.execute(create_owners_sql)

In [84]:
# ok, let's add some data:
cursor.execute("DELETE FROM Levels")
cursor.execute("INSERT INTO Levels (id, name, short_name) VALUES (1, 'Postgraduate', 'PG' )")
cursor.execute("INSERT INTO Levels (id, name, short_name) VALUES (2, 'Undergraduate', 'UG')")
cursor.execute("SELECT * FROM Levels").fetchall()

[(1, 'Postgraduate', 'PG'), (2, 'Undergraduate', 'UG')]

In [85]:
# ok, let's add some data:
cursor.execute("DELETE FROM Pets")
cursor.execute("INSERT INTO Pets (id, name) VALUES (1, 'cat')")
cursor.execute("INSERT INTO Pets (id, name) VALUES (2, 'dog')")
cursor.execute("INSERT INTO Pets (id, name) VALUES (3, 'fish')")
cursor.execute("SELECT * FROM Pets").fetchall()

[(1, 'cat'), (2, 'dog'), (3, 'fish')]

In [86]:
# ok, let's add some data:
cursor.execute("DELETE FROM Students")
cursor.execute("INSERT INTO Students (id, name, age, level_id) VALUES (1, 'Pat', 34, 1)")
cursor.execute("INSERT INTO Students (id, name, age, level_id) VALUES (2, 'Natasha', 64, 2)")
cursor.execute("INSERT INTO Students (id, name, age, level_id) VALUES (3, 'Pip', 54, 2)")
cursor.execute("INSERT INTO Students (id, name, age, level_id) VALUES (4, 'Jo', 37, 1)")
cursor.execute("SELECT * FROM Students").fetchall()

[(1, 'Pat', 34, 1), (2, 'Natasha', 64, 2), (3, 'Pip', 54, 2), (4, 'Jo', 37, 1)]

In [89]:
# ok, let's add some data:
cursor.execute("DELETE FROM Owners")
# Pat has two cats and a dog
cursor.execute("INSERT INTO Owners (student_id, pet_id) VALUES (1, 1)")
cursor.execute("INSERT INTO Owners ( student_id, pet_id) VALUES (1, 1)")
cursor.execute("INSERT INTO Owners ( student_id, pet_id) VALUES (1, 2)")
# Natasha has three cats
cursor.execute("INSERT INTO Owners (student_id, pet_id) VALUES (2, 1)")
cursor.execute("INSERT INTO Owners ( student_id, pet_id) VALUES (2, 1)")
cursor.execute("INSERT INTO Owners ( student_id, pet_id) VALUES (2, 1)")
# Pip and Jo have a fish and cat each
cursor.execute("INSERT INTO Owners (student_id, pet_id) VALUES (3, 1)")
cursor.execute("INSERT INTO Owners (student_id, pet_id) VALUES (3, 3)")
cursor.execute("INSERT INTO Owners ( student_id, pet_id) VALUES (4, 1)")
cursor.execute("INSERT INTO Owners ( student_id, pet_id) VALUES (4, 3)")

cursor.execute("SELECT * FROM Owners").fetchall()

[(1, 1, 1),
 (2, 1, 1),
 (3, 1, 2),
 (4, 2, 1),
 (5, 2, 1),
 (6, 2, 1),
 (7, 3, 1),
 (8, 3, 3),
 (9, 4, 1),
 (10, 4, 3)]

Let's run a very simple query:

bewlo `*` means **all columns**, but you could also specify only the columns you want. It would look like this `SELECT Name, Age FROM Students`

In [90]:
cursor.execute("SELECT * FROM Students WHERE Students.level_id = 1")
rows = cursor.fetchall()
rows
# have a peak at the data above, where we created tables data, what is going on here?

[(1, 'Pat', 34, 1), (4, 'Jo', 37, 1)]

And here is a more complex query with joins. if you have not seen it before, tripple " creates a special type of string where you can use new lines. This is very useful for readability!


In [91]:
sql_query = """
SELECT Students.name, Pets.name 
FROM Pets
LEFT JOIN Owners ON Pets.id = Owners.pet_id
LEFT JOIN Students ON Students.id = Owners.student_id
JOIN Levels ON Students.level_id = Levels.id
WHERE Levels.short_name = 'UG'
"""
cursor.execute(sql_query)
rows = cursor.fetchall()
rows
# have a peak at the data above, where we created tables data, what is going on here?

[('Natasha', 'cat'),
 ('Natasha', 'cat'),
 ('Natasha', 'cat'),
 ('Pip', 'cat'),
 ('Pip', 'fish')]

#### But how do we then turn this database result info dataframes?

Luckilly (as always!) Pandas has a function for that! We pass into that function the full SQL string, and the `connection variable` (not the cursor! Pandas will operate the cursor by itself).

Have a look below how it will look like

In [92]:
sql_query = """
SELECT Students.name, Pets.name
FROM Pets
LEFT JOIN Owners ON Pets.id = Owners.pet_id
LEFT JOIN Students ON Students.id = Owners.student_id
JOIN Levels ON Students.level_id = Levels.id
WHERE Levels.short_name = 'UG'
"""

df = pd.read_sql_query(sql_query, connection)
df.head()

,name,name
0,Natasha,cat
1,Natasha,cat
2,Natasha,cat
3,Pip,cat
4,Pip,fish


Oh no! but both columns have the same name! (`'name'`). That's really not useful in Pandas!

**Giving nicknames to tables and columns** is a very useful technique where you can give a short name to things, like

```
SELECT department.name dept_name 
FROM HospitalDepartment department
```

instead of 

```
SELECT HospitalDepartment.name 
FROM HospitalDepartment
```

This is especially useful when we want to have meaningful column names in pandas. 

In [93]:
sql_query = """
SELECT Students.name student_name, Pets.name  pet_name
FROM Pets
LEFT JOIN Owners ON Pets.id = Owners.pet_id
LEFT JOIN Students ON Students.id = Owners.student_id
JOIN Levels ON Students.level_id = Levels.id
WHERE Levels.short_name = 'UG'
"""

df = pd.read_sql_query(sql_query, connection)
df.head()

,student_name,pet_name
0,Natasha,cat
1,Natasha,cat
2,Natasha,cat
3,Pip,cat
4,Pip,fish


Finally, when you are done working with a database, it is a good idea to close the connections. This is very important when many users access the databse at the same time, as it might have a limit of maximum consecutive connections.

In [94]:
# close conenction
conn.close()

<a id="api-calls"></a>
# 5. Making API Calls

API (Application Programming Interface) is an alternative way of accessing data and can be very powerful. There are a plethora of APIs available on the internet - some free, some that you have to pay for and some that require you to sign up for a key.

NHS Scotland Opendata has most of its datasets available as APIs. You can search for a dataset and click on the green "Data API" button to get the url. <a href="https://www.opendata.nhs.scot/organization/isd">click here for PHS Health and care datasets</a>

## Requests and Responses

When we create an API call in Python, we are making a **request**. Most of the time, this is a GET request i.e. we're asking the URL to get us some data. The server that lives at the URL address will respond with a **response**.

## What is a URL?

In order to request data from an API we need to know the URL (Uniform Resource Locator). This is essentially the address of a resource on the internet. Think of it like a street address for a website or a specific file. It tells us where to find what you're looking for. 


## JSON again!

Usually, APIs will respond with the data in JSON format. The tricky part, as ever with JSON, is examining the structure of the data so that you can drill down into the data to get the parts that are relevant.

In [ ]:
# Let's use an example of a fun API for illustration purposes - the Pokemon API (Pokemon are little cartoon characters)
# Here's the URL for accessing the data of the top 20 Pokemon

url = 'https://pokeapi.co/api/v2/pokemon'

# https: This is the protocol used to access the data (hypertext transfer protocol secure)
# pokeapi.co: This is the domain name of the website hosting the API
# api/v2: This is the path to the API endpoint
# pokemon: This is the resource we are accessing
# you can think of it as a bit like a file system on your computer. Each part of the URL is like a folder or file.


# we can use the requests library to make a GET request to the URL
import requests

pokemon_df = None

try:
    response = requests.get(url)  # Make the GET request
    response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)

    pokemon_data = response.json()  # Parse the JSON response
    pokemon_df  = pd.DataFrame.from_dict(pokemon_data['results'])

except requests.exceptions.RequestException as e:
    print(f"Error: {e}")

pokemon_df


In [ ]:
# Now we have a DataFrame with the names and URLs of the first 20 Pokemon!
# We could loop through the names and display them
for counter, name in enumerate(pokemon_df['name'], start=1): # Use enumerate to get a counter AND name
    print(f"Pokemon number {counter}: {name}")      

In [ ]:

# Or we can use the URL for the "bulbasaur" Pokemon to get more information about it
bulbasaur_url = pokemon_df.loc[pokemon_df['name'] == 'bulbasaur', 'url']
bulbasaur_url

# We can use the requests library to make a GET request to the URL
bulbasaur_data = None

try:
    response = requests.get(bulbasaur_url.iloc[0])  # Make the GET request
    response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)

    bulbasaur_data = response.json()

except requests.exceptions.RequestException as e:
    print(f"Error: {e}") 

# Let's see what we got. It's a dict!
print(f"Bulbasaur data type: {type(bulbasaur_data)}")

# Let's see what keys we have in the dict
bulbasaur_data.keys()

# We could create a DataFrame from the abilities key for example
bulbasaur_abilities_df = pd.DataFrame(bulbasaur_data['abilities'])
bulbasaur_abilities_df

## Final Tasks:

1. Practice unbundling data. Take a variables you created before, and undbundle/parse them the same way we unbundled hospital staff in examples above.
2. Look at two ways of acquiring the same data (api request, and SQL reqiest), and find simmilarities and differences between them.


### Task 1: `bulbasaur_data` variable

In [ ]:
# you can code here, here's a starting variable. We care about the abilities of this pokemon.
import pprint as pp

pp.pprint(bulbasaur_data)

### Challenge 2: getting online data via simple API and via online SQL

There is no 'task as such' below. But have a look at what the code does, and see how much you understand.



#### Part 1: Simple API

Let's make an API request to get the data from the NHS Scotland website about nursing home residents
The data is in json format.
We can use our json knowledge to turn the correct part of the json data into a DataFrame

In [ ]:
import requests
import pprint as pp

url = 'https://www.opendata.nhs.scot/api/3/action/datastore_search?resource_id=139f61d8-a87d-419d-b7af-31f555a60c89'  

nursing_home_residents_df = None

try:
    response = requests.get(url)  # Make the GET request
    response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)

    data = response.json()  # Parse the JSON response
    # HERE WRITE YOUR CODE
    nursing_home_residents_df  = pd.DataFrame(data['result']['records']) # We are interested in the 'records' part of the data

except requests.exceptions.RequestException as e:
    print(f"Error: {e}")


# to start you off let's print the data format.
# pp.pprint(data['result']['records']) #if you're interested how raw data looks, uncomment this
nursing_home_residents_df

#### Part 2: Simple online sql

This is a very interesting mix of API and SQL: you can pass in SQL code as part of your url, like

`https://somewebsite.com?sql=SELECT name FROM students`

We will build the SQL, then send it to the API to get interpreted.


In [ ]:

sql = f'SELECT "HBName", "HB" FROM "652ff726-e676-4a20-abda-435b98dd7bdc"'
# sql = f'SELECT * FROM "652ff726-e676-4a20-abda-435b98dd7bdc"' # if you want all columns, use *

website_url = "https://www.opendata.nhs.scot/api/3/action/datastore_search_sql"

final_url = f"{website_url}?sql={sql}"

response = requests.get(final_url)
# pp.pprint(response.json()) #if you're interested how raw data looks, uncomment this
hb_lookup = pd.DataFrame(response.json()['result']['records'])
hb_lookup


## ⭐️⭐️⭐️💥 What you learned in this session: Three stars and a wish 
**In your own words** write in your Learn diary:

- 3 things you yould like to remember from this badge
- 1 thing you wish to understand better in the future or a question you'd like to ask
